<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [15]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
import pytz
import numpy as np
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [16]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# with open('../data/hopsworks-api-key.txt', 'r') as file:
#     os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1159326
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "austria", "city": "vienna", "street": "vienna-taborstra\\u00dfe-glockengasse", "aqicn_url": "https://api.waqi.info/feed/@2860", "latitude": 48.21, "longitude": 16.37}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [ ]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)


weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)


air_quality_lag_fg = fs.get_feature_group(
    name='air_quality_with_rolling_window_3_days',
    version=1,
)


---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [18]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,13.0,austria,vienna,vienna-taborstraße-glockengasse,2024-11-18,https://api.waqi.info/feed/@2860


In [19]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [20]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 48.0°N 16.5°E
Elevation 194.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-18,9.85,0.0,1.835647,191.309891,vienna
1,2024-11-19,7.40,0.0,11.966954,136.218796,vienna
2,2024-11-20,5.10,0.1,17.610588,310.855286,vienna
3,2024-11-21,3.85,0.0,24.826952,286.858490,vienna
4,2024-11-22,3.55,0.0,26.987997,313.919159,vienna
5,2024-11-23,5.40,0.0,28.412645,292.340698,vienna
6,2024-11-24,6.25,0.0,10.245780,108.435043,vienna
7,2024-11-25,6.60,0.0,16.299694,133.210114,vienna
8,2024-11-26,5.65,0.0,14.007655,334.093536,vienna
9,2024-11-27,6.85,0.0,11.570515,354.644257,vienna


In [21]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [22]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2024-11-18 22:25:43,978 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1159326/fs/1150029/fg/1338743


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1159326/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1cf621c8f10>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 662538
         }
       },
       "result": {
         "observed_value": 12.999999046325684,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-18T09:25:43.000978Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations":

In [ ]:

# Calculate date range for the last 3 days
three_days_ago = datetime.datetime.now() - datetime.timedelta(days=3)
three_days_ago = three_days_ago.replace(hour=0, minute=0, second=0, microsecond=0, tzinfo=pytz.UTC)

# Read the air quality data into a DataFrame
air_quality_df = air_quality_fg.read()

# Ensure 'date' column is in datetime format, if not convert it
if not pd.api.types.is_datetime64_any_dtype(air_quality_df['date']):
    air_quality_df['date'] = pd.to_datetime(air_quality_df['date'])

# Filter the data for the last 3 days
three_days_data = air_quality_df[air_quality_df['date'] >= three_days_ago]

# Sort by date for accurate rolling computation
three_days_data = three_days_data.sort_values('date')


# Calculate the rolling mean for a 3-day window for 'pm25'
three_days_data['pm25_prev'] = three_days_data['pm25'].mean()


# Check if any records exist for the last 3 days
if not three_days_data.empty:
    print("There are records from the last 3 days:")
    # If yes, prepare the data for insertion into the lagged feature group
    prev_data = three_days_data['pm25_prev'].iloc[-1]
    data_to_insert = pd.DataFrame({
        'pm25':[np.float32(aq_today_df['pm25'].values[0])],
        'pm25_prev': np.double([prev_data]),
        'date': [datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0, tzinfo=pytz.UTC)],
        'country': ['austria'],
        'city': ['vienna'],
        'street': ['vienna-taborstraße-glockengasse'],
        'url': ['https://api.waqi.info/feed/@2860']
    })
    
    # Insert the DataFrame into the feature group
    air_quality_lag_fg.insert(data_to_insert)
    print(data_to_insert)
else:
    print("No records for the last 3 days.")



Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.40s) 
There are records from the last 3 days:


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_with_rolling_window_3_days_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1159326/jobs/named/air_quality_with_rolling_window_3_days_1_offline_fg_materialization/executions
   pm25  pm25_prev                      date  country    city  \
0  13.0       36.5 2024-11-18 00:00:00+00:00  austria  vienna   

                            street                               url  
0  vienna-taborstraße-glockengasse  https://api.waqi.info/feed/@2860  


In [24]:
# Insert new data
weather_fg.insert(daily_df)

2024-11-18 22:26:13,803 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1159326/fs/1150029/fg/1337716


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1159326/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1cf627b7df0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 661514
         }
       },
       "result": {
         "observed_value": 1.8356469869613647,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-18T09:26:13.000803Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
